In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np

In [3]:
import keras.backend as K

Using TensorFlow backend.


In [4]:
from keras.utils import to_categorical

In [5]:
from keras.layers import Input, Conv2D, Dense, Flatten

In [6]:
from keras.models import Model

In [7]:
from keras.optimizers import SGD

In [8]:
from opt.prune_mask import PruneWeights

In [9]:
X_train = np.load("../data/X_train.npy")
y_train = np.load("../data/y_train.npy")
X_test = np.load("../data/X_test.npy")
y_test = np.load("../data/y_test.npy")

In [10]:
X_train = X_train / 127.5 - 1
X_test = X_test / 127.5 - 1

In [11]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [12]:
inputs = Input((32, 32, 3))

In [13]:
x = Conv2D(64, 3, activation="relu")(inputs)

In [14]:
x = Conv2D(64, 3, activation="relu")(x)

In [15]:
x = Flatten()(x)

In [16]:
outputs = Dense(10, activation="softmax")(x)

In [17]:
model = Model(inputs=inputs, outputs=outputs)

In [18]:
sgd = SGD(lr=1e-4, momentum=0.9, nesterov=True)

In [19]:
model.compile(sgd, loss="categorical_crossentropy")

In [20]:
sess = K.get_session()

In [21]:
callbacks = []
callbacks.append(PruneWeights(model, sess))

In [29]:
model.fit(x=X_train[:100], y=y_train[:100], batch_size=32, epochs=20, verbose=1, callbacks=callbacks, validation_data=(X_test, y_test))

Train on 100 samples, validate on 10000 samples
Epoch 1/20
Prune
100/100 [==============================] - 1s 8ms/step - loss: 2.1409 - val_loss: 2.3109
Epoch 2/20
100/100 [==============================] - 1s 8ms/step - loss: 2.1345 - val_loss: 2.3110
Epoch 3/20
100/100 [==============================] - 1s 8ms/step - loss: 2.1285 - val_loss: 2.3110
Epoch 4/20
100/100 [==============================] - 1s 8ms/step - loss: 2.1222 - val_loss: 2.3107
Epoch 5/20
100/100 [==============================] - 1s 8ms/step - loss: 2.1163 - val_loss: 2.3104
Epoch 6/20
100/100 [==============================] - 1s 8ms/step - loss: 2.1097 - val_loss: 2.3104
Epoch 7/20
100/100 [==============================] - 1s 8ms/step - loss: 2.1030 - val_loss: 2.3115
Epoch 8/20
100/100 [==============================] - 1s 8ms/step - loss: 2.0963 - val_loss: 2.3129
Epoch 9/20
100/100 [==============================] - 1s 8ms/step - loss: 2.0890 - val_loss: 2.3146
Epoch 10/20
100/100 [=========================

In [40]:
for lay in model.layers:
    cls = lay.__class__
    if cls == Conv2D:
        w = lay.get_weights()[0]
        zero_c = np.sum(np.isclose(w, 0))
        print(zero_c)
        print(len(w.ravel()))

173
1728
3687
36864
